<a href="https://colab.research.google.com/github/rybread1/coronavirus-tweet-nlp/blob/master/coronavirus_tweet_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 890kB 4.6MB/s 
     |████████████████████████████████| 3.0MB 25.0MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 
     |████████████████████████████████| 1.1MB 40.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=dafc0e48c2d55aac51348f167c8bb27b0b4f8ec3f005338b6c3b647fdf388d7a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import math
import seaborn as sn
import tensorflow as tf
import tensorflow_datasets as tfds
from bs4 import BeautifulSoup
from sklearn import metrics as sk_metrics
import nltk
import re

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
folder = '/content/gdrive/My Drive/corona-tweet-sent'
train_f = 'Corona_NLP_train.csv'
test_f = 'Corona_NLP_test.csv'

df_train = pd.read_csv(os.path.join(folder, train_f), engine='python')
df_test = pd.read_csv(os.path.join(folder, test_f), engine='python')

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [6]:
df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [17]:
y_map = {
    'Positive': 0, 
    'Negative': 1, 
    'Neutral': 2, 
    'Extremely Positive': 0, 
    'Extremely Negative': 1
}

df_train['Sentiment_Cat'] = df_train['Sentiment'].map(y_map)
df_test['Sentiment_Cat'] = df_test['Sentiment'].map(y_map)

In [18]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def clean_text(data):
    data = data.lower()
    data = re.sub(r'([^a-zA-Z\s])', '', data)
    data = data.split()
    temp = []
    for i in data:
        if i not in stop_words:
            temp.append(i)
    data = ' '.join(temp)
    return data

df_train['tweet_cleaned'] = df_train['OriginalTweet'].apply(clean_text)
df_test['tweet_cleaned'] = df_test['OriginalTweet'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
x_train, y_train = df_train['tweet_cleaned'].values, np.array(df_train['Sentiment_Cat'], dtype=np.int64)
x_test, y_test = df_test['tweet_cleaned'].values, np.array(df_test['Sentiment_Cat'], dtype=np.int64)

In [20]:
from tokenizers import BertWordPieceTokenizer
import transformers

In [21]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased' , lower=True)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [22]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=200):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [23]:
max_len=50

x_train_enc = fast_encode(x_train, fast_tokenizer, maxlen=max_len)
x_test_enc = fast_encode(x_test, fast_tokenizer, maxlen=max_len)

In [27]:
NUM_CLASSES = df_train['Sentiment_Cat'].nunique()

bert_transformer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')

input = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
sequence_output = bert_transformer(input)[0]
cls_token = sequence_output[:, 0, :]
output = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(cls_token)

model = tf.keras.Model(inputs=input, outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=7e-6), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [28]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
tf_distil_bert_model_2 (TFDi ((None, 50, 768),)        66362880  
_________________________________________________________________
tf_op_layer_strided_slice_2  [(None, 768)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 2307      
Total params: 66,365,187
Trainable params: 66,365,187
Non-trainable params: 0
_________________________________________________________________


In [29]:
# call backs
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True
)

history = model.fit(
    x_train_enc,
    y_train,
    batch_size=32,
    validation_data=(x_test_enc, y_test),
    epochs=50,
    callbacks=[early_stop_callback])

Epoch 1/50
644/644 [==============================] - 224s 348ms/step - loss: 0.6677 - accuracy: 0.7190 - val_loss: 0.5366 - val_accuracy: 0.8002
Epoch 2/50
644/644 [==============================] - 222s 345ms/step - loss: 0.4653 - accuracy: 0.8319 - val_loss: 0.4923 - val_accuracy: 0.8189
Epoch 3/50
644/644 [==============================] - 222s 344ms/step - loss: 0.3937 - accuracy: 0.8618 - val_loss: 0.4566 - val_accuracy: 0.8418
Epoch 4/50
644/644 [==============================] - 222s 344ms/step - loss: 0.3411 - accuracy: 0.8848 - val_loss: 0.4546 - val_accuracy: 0.8399
Epoch 5/50
644/644 [==============================] - 222s 344ms/step - loss: 0.3030 - accuracy: 0.8990 - val_loss: 0.4593 - val_accuracy: 0.8481
Epoch 6/50
644/644 [==============================] - 222s 345ms/step - loss: 0.2666 - accuracy: 0.9129 - val_loss: 0.4489 - val_accuracy: 0.8507
Epoch 7/50
644/644 [==============================] - 222s 344ms/step - loss: 0.2362 - accuracy: 0.9224 - val_loss: 0.4644 -